In [84]:
import os
import sys
import urllib.request
import json
import time
client_id = "CPNilOaVBk_G8vJBdkOQ"
client_secret = "m4f9d4EL6V" 
quote = input("검색어를 입력해주세요.: ") 
encText = urllib.parse.quote(quote)
display_num = str(100) 
    

dates = []
# titles = []
# sections = []
# contents = []
# links2 = []
for i in range(1, 1001):
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&display=100&start=i"
    
    
    
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    
#     for a in data['items']:
#         dates.append(a['postdate'])
#         titles.append(a['title'])
#         sections.append("블로그")
#         links2.append(a['link'])
    
    if(rescode==200):
        response_body = response.read()
#         print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
    
    time.sleep(2)
    body = response_body.decode('utf-8')
    res = json.loads(body)
    
    
#     print('\\\\\\\\\\\\\ dATE......\\\\\\\\\\\\')
    for a in res['items']:
        dates.append(a['postdate'])
        print(a['postdate'])
    
#     print(dates)
#     print(body)
    
    # 불필요한 ""(큰따옴표)지워주기
    body = body.replace('"','')
    
#     #블로그 제목만 뽑기
    import re
    titles = re.findall('title: (.*?),\nlink', body) #\n은 엔터의 의미
    print('<<제목 모음>>')
    print(titles)
    print('총 제목 수: ',len(titles),'개')#제목갯수확인
#     "title": "화이자 부스터샷, <b>코로나 백신</b> 3차 접종 몸상태 후기",
#     "link": "https:\/\/blog.naver.com\/bom25bom?Redirect=Log&logNo=222668141935",
#     "bloggerlink": "https://blog.naver.com/bom25bom",
#     "postdate": "20220309"

    #블로그 링크들 추출
    links = re.findall('link: (.*?),\ndescription',body)
    print('\n<<링크 모음>>')
    print(links)
    print('총 링크 수: ',len(links),'개')#링크갯수확인
    
    # 링크를 다듬기 (필요없는 부분 제거 및 수정)
    blog_links = []
    sections = []
    for i in range(100):
        sections.append("블로그")

    for i in links:
        a = i.replace('\\','')
        b = a.replace('?Redirect=Log&logNo=','/')
        blog_links.append(b)

    print(blog_links)
    print('생성된 링크 갯수:',len(blog_links),'개')
    
    #본문 크롤링
    import time
    from selenium import webdriver
    from bs4 import BeautifulSoup
    from selenium.common.exceptions import NoSuchElementException

    driver = webdriver.Chrome(os.path.abspath('chromedriver')) 
    driver.implicitly_wait(3)
    
    #블로그 링크 하나씩 불러서 크롤링
    contents = []
    for i in blog_links:
        #블로그 링크 하나씩 불러오기
        driver.get(i)
        time.sleep(1)

        try:
#             driver.switch_to.frame("mainFrame")
            #블로그 안 본문이 있는 iframe에 접근하기

            element = driver.find_element_by_id("mainFrame") #iframe 태그 엘리먼트 찾기
            driver.switch_to.frame(element) #프레임 이동
            #본문 내용 크롤링하기
            a = driver.find_element_by_css_selector('div.se-main-container').text
            contents.append(a)

        # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
        except NoSuchElementException:
            print("Exception 발생")
        #print(본문: \n', a)




    driver.quit() #창닫기
    print("<<본문 크롤링이 완료되었습니다.>>")
    
    dif = len(titles) - len(contents)
    print(dif)
    for i in range(dif):
        titles.pop(i)
        sections.pop(i)
        dates.pop(i)
    
    print(sections)
    print(dates)
    print(titles)
    print(contents)
    print(len(sections))
    print(len(dates))
    print(len(titles))
    print(len(contents))

    #제목, 블로그링크, 본문내용 Dataframe으로 만들기
    import pandas as pd
    df = pd.DataFrame({'SECTION':sections , 'DATE':dates ,'TITLE':titles,'CONTENT':contents})
    df

검색어를 입력해주세요.: 코로나 백신


HTTPError: HTTP Error 400: Bad Request

In [85]:
df

,SECTION,DATE,TITLE,CONTENT
0,블로그,20211217,화이자 <b>코로나 백신</b> 3차 접종 후기!,Booster shot \n봄빛씨입니다. \n며칠 전 화이자 부스터샷 접종 하고 왔...
1,블로그,20220226,화이자 3차 후기 : 일본에서 <b>코로나 백신</b> 접종 완료,오늘 코로나 백신 3차 맞고 왔습니다. 주변에서 몸도 약한 사람이 멀 그리 빨리 맞...
2,블로그,20210929,<b>코로나백신</b> 모더나 1차 접종 부작용 후기(타이레놀 복용법),청소년 백신 예약 및 접종. 2차 예약 변경 방법(코로나19)\n청소년 백신 접종에...
3,블로그,20220221,<b>코로나 백신</b> 화이자 부작용... 혈소판증가증 혈전 | 백신 일기,코로나 백신 접종 : 화이자 3차 후기\n20220226\n안녕하세요 라뿡맘입니다....
4,블로그,20220311,"일본/도쿄 일상) 3월 : <b>코로나 백신</b> 화이자 3차 접종, EMS 택...",코로나 확진자의 백신 2차 접종 후기\n개인적인 증상을 적은 일기 글입니다\n의식의...
...,...,...,...,...
82,블로그,20211209,<b>코로나백신</b>접종피해자 현실 촛불집회,정부가 5~11세 어린이에 대한 신종 코로나바이러스 감염증(코로나19) 백신접종을 ...
83,블로그,20220109,모더나로 <b>코로나</b> 3차 <b>백신</b> 접종을 했는데...,재택치료요령\n재생 수\n15\n0\n00:00 | 00:44\n화질 선택 옵션\n...
84,블로그,20220113,"씨티씨백, 수출용 반려동물 <b>코로나 백신</b> 품목허가",2차 맞은지 6달 다되어서야 3차를 맞았다.\nhttps://blog.naver.c...
85,블로그,20220115,임산부 <b>코로나 백신</b> 접종 일정 알아보세요,코로나 백신 쉐딩 진짜 있습니다\n이것은 뇌피셜이 아닙니다.\n대한민국 식품의약품안...


In [81]:
df.to_csv('blog.csv')

In [30]:
#[CODE1 ]
def get_request_url(url):
    print("####>get_request_url(url)###")

    req = urllib.request.Request(url) #url정보 받아옴
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200: # 200 : 성공적으로 수행되었다는 뜻(OK의 코드값 : 200)
            print("get_request_url, [%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("get_request_url, [%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [31]:
#[CODE2 ]
def getNaverSearchResult(sNode, search_text, page_start, display):
    print("####>getNaverSearchResult()###")

    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)

    url = base + node + parameters #위의 정보로 url 완성

    retData = get_request_url(url)

    if(retData == None):
        print("<getNaverSearchResult, retData> 없음, ", retData)
        return None
    else:
        print("<getNaverSearchResult, retData > = ", retData)
    return json.loads(str(retData))

In [32]:
#[CODE3 ]
def getPostData(post, jsonResult):
    print("####>getPostData()###")

    title = post['title']
    description = post['description']
    org_link = post['bloggerlink']
    link = post['link']
    pDate = post['postdate']
    bloggername = post['bloggername']

    jsonResult.append({'title':title, 'description':description,
    'org_link':org_link, 'link':link,
    'pDate':pDate, 'bloggername':bloggername})
    return

In [35]:
import datetime

print("####>main()###")
jsonResult = []
sNode = 'blog'
search_text = '코로나 백신'
display_count = 100 # display : 한번에 100개 => 1000개

nStart = 1

jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)
print('main, jsonSearch =', jsonSearch)

while((jsonSearch != None) and (jsonSearch['display'] != 0) and (nStart + display_count < 1000)):
    for post in jsonSearch['items']: # 'items' key에 기사가 저장되어 있음
        getPostData(post, jsonResult)

        try:
            nStart = jsonSearch['start'] + jsonSearch['display']
            print("jsonSearch['start']: ", jsonSearch['start'])
            print("jsonSearch['display']: ", jsonSearch['display'])
            print("nStart: ", nStart)
            jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)

            with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf-8') as outfile:
                retJson = json.dumps(jsonResult,
                indent=4, sort_keys=True,
                ensure_ascii=False)
                outfile.write(retJson)

                print('%s_naver_%s.json SAVED' % (search_text, sNode))
        except:
            print("실패")

####>main()###
####>getNaverSearchResult()###
####>get_request_url(url)###
get_request_url, [2022-03-15 02:33:36.779706] Url Request Success
<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:36 +0900",
"total": 1470715,
"start": 1,
"display": 100,
"items": [
{
"title": "화이자 부스터샷, <b>코로나 백신</b> 3차 접종 몸상태 후기",
"link": "https:\/\/blog.naver.com\/bom25bom?Redirect=Log&logNo=222668141935",
"description": "원래 <b>코로나 백신</b> 3차는 맞기 싫어서 백신패스가 없어지길 간절히 바랐는데 마침 3월 초부터 없어져서 사전 예약을 한번 취소했었어요. 그런데 확진자들이 너무 많이 나오는 데다 부스터샷을 기본 접종으로... ",
"bloggername": "봄빛씨는 언제나 봄날",
"bloggerlink": "https://blog.naver.com/bom25bom",
"postdate": "20220309"

},
{
"title": "화이자 <b>코로나 백신</b> 3차 접종 후기!",
"link": "https:\/\/blog.naver.com\/aldus_06?Redirect=Log&logNo=222598074981",
"description": "오늘 <b>코로나 백신</b> 3차 맞고 왔습니다. 주변에서 몸도 약한 사람이 멀 그리 빨리 맞냐며 걱정 섞인... 그래도 혹시 모르니 꼭 코로나19 백신 맞기 전 아세트아미노펜 성분이 포함된 해열진통제를 준비해 놓는... ",
"bloggername": "텃밭연구소",
"bloggerlink": "https://blog.naver.com/aldus_06"

<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:37 +0900",
"total": 1470715,
"start": 101,
"display": 100,
"items": [
{
"title": "# 3 _ <b>코로나 백신</b> 접종 최최종!",
"link": "https:\/\/blog.naver.com\/juheeo9o2?Redirect=Log&logNo=222652383775",
"description": "드디어 <b>코로나 백신</b> 3차 접종을 끝냈다. 딱히 접종을 미뤘다거나 그랬던 건 아니었고, 10월에 2차를... 3차까지 접종해도 코로나에 걸릴수도 있겠지만 그러면 뭐하러 백신을 맞아? 라고 묻는 사람이 있을 수도... ",
"bloggername": "반짝반짝 빛나는 ♥",
"bloggerlink": "https://blog.naver.com/juheeo9o2",
"postdate": "20220220"

},
{
"title": "<b>코로나 백신</b> 비접종자 조롱하는 스탠드업 코메디 공연 중... ",
"link": "https:\/\/blog.naver.com\/peace_2000?Redirect=Log&logNo=222645905478",
"description": "https://m.youtube.com/watch?v=akFILtJC-Xg&amp;feature=emb_title &quot;나는 <b>백신</b>을 맞았고, 2차 접종에 부스터 샷도 맞았고... 독감 주사, 대상포진 주사도 맞았고, <b>코로나</b>에 감염된 적 없어. 예수님이... ",
"bloggername": "관심가는 대로",
"bloggerlink": "https://blog.naver.com/peace_2000",
"postdate": "20220212"

},
{
"title": "<b>코로나</b>19 <b>백신</b> 특허 면제? <b>백신</b> 공급 확

get_request_url, [2022-03-15 02:33:37.291037] Url Request Success
<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:37 +0900",
"total": 1470715,
"start": 301,
"display": 100,
"items": [
{
"title": "18세~49세 <b>코로나 백신</b>10부제 일정, 예약 방법 정리",
"link": "https:\/\/blog.naver.com\/alswl09100?Redirect=Log&logNo=222451174186",
"description": "드디어 18세~49세 <b>코로나 백신</b> 10부제 예약 일정이 확정되었네요. 젊은 사람들부터 맞아야 확산되는 걸... 여기저기서 이 전염병으로 인해 고통받고 있는데 하루 빨리 백신이라도 맞아서 경기가 활성화 되는... ",
"bloggername": "재테크하는 제인",
"bloggerlink": "https://blog.naver.com/alswl09100",
"postdate": "20210730"

},
{
"title": "<b>코로나</b>19 <b>백신</b>과 피부과 약물처방",
"link": "https:\/\/blog.naver.com\/divadance?Redirect=Log&logNo=222385755319",
"description": "<b>코로나 백신</b>과 관련하여 피부과 처방약물과 백신의 효과 및 부작용에 대한 질문을 많이 하시는데요. 이를 간략히 정리해드리겠습니다. 1. 피부과 항생제 항염효과를 갖고 있는 테트라사이클린 계통의 여드름... ",
"bloggername": "피부과전문의 김지영원장의 닥터센서티브",
"bloggerlink": "https://blog.naver.com/divadance",
"postdate": "20210605"

},
{
"title": "<b>코로나<

get_request_url, [2022-03-15 02:33:37.633714] Url Request Success
<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:37 +0900",
"total": 1470715,
"start": 401,
"display": 100,
"items": [
{
"title": "방역패스의 문제점과 <b>코로나</b> 부스트샷 <b>백신</b>부작용 &quot;<b>백신</b>부작용... ",
"link": "https:\/\/blog.naver.com\/aroma_sc?Redirect=Log&logNo=222598446238",
"description": "코로나 2차 백신부작용 아나필락시스 발생으로 응급실로 실려간 경험이 있는데요. 아나필락시스 발생을... <b>코로나백신</b>(화이자)을 맞고 2번 모두 불수의적인 근수축을 보였는데 그 원인을 알수 없는 상황입니다.... ",
"bloggername": "아로마_에스씨",
"bloggerlink": "https://blog.naver.com/aroma_sc",
"postdate": "20211218"

},
{
"title": "<b>코로나백신</b> 누구는 접종하고 누구는 안 맞는가? <b>코로나백신</b>... ",
"link": "https:\/\/blog.naver.com\/kpg8004?Redirect=Log&logNo=222661506363",
"description": "코로나19백신 접종으로 부작용이 따른다고 하지만 그래도 다수의 국민이 <b>코로나백신</b> 접종후에 잘 살고 있지 않은가. 접종 거부자들의 논리나 내 건강도 좋고 다 좋다 그렇지만 ㆍㆍㆍㆍㆍ 더 말하면 그렇지... ",
"bloggername": "땅끝햇빛담은농원",
"bloggerlink": "https://blog.naver.com/kpg8004",
"postdate": "20220302"

},


get_request_url, [2022-03-15 02:33:38.017452] Url Request Success
<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:38 +0900",
"total": 1470715,
"start": 501,
"display": 100,
"items": [
{
"title": "중학생 <b>코로나</b>19 화이자 <b>백신</b> 2차 접종.",
"link": "https:\/\/blog.naver.com\/jirisan72?Redirect=Log&logNo=222594024549",
"description": "다만 위드코로나 시행후 코로나 환자 급증!!! 백신도 중요하지만 결국 대면과 비대면이 코로나 유행을... 하물며 급하게 만들어낸 <b>코로나 백신</b>은... 백신을 강요할 순 없지만 건강상의 이유와 불신으로 맞지 않은... ",
"bloggername": "지리산과 검은별 ★~",
"bloggerlink": "https://blog.naver.com/jirisan72",
"postdate": "20211213"

},
{
"title": "<b>코로나백신</b> 3차(부스터샷) 예약했어요.",
"link": "https:\/\/blog.naver.com\/longgutwon?Redirect=Log&logNo=222594949736",
"description": "있는 중 다시금 확산이 되어 백신패스도 도입하고 그르든데.. 그거때매 말도 많죠? 어쨌든! 다가오는 3차 접종도 화이팅하며! 접종 후 후기도 올려볼께요~^^ #<b>코로나백신</b>접종 #백신부스터샷 #코로나3차예약",
"bloggername": "엉.뚱.매.력.날.짱. :)",
"bloggerlink": "https://blog.naver.com/longgutwon",
"postdate": "20211214"

},
{
"title": "<b>코로나 백신</b>

get_request_url, [2022-03-15 02:33:38.462280] Url Request Success
<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:38 +0900",
"total": 1470715,
"start": 601,
"display": 100,
"items": [
{
"title": "겨드랑이멍울 원인과 <b>코로나 백신</b> 관련",
"link": "https:\/\/blog.naver.com\/msent2287?Redirect=Log&logNo=222524478441",
"description": "<b>코로나 백신</b> 부작용?? 요즘 또 한창 백신을 맞는 주기라 다양한 백신의 부작용이 나타났다는 신고가 들어오곤 하는데 겨드랑이멍울도 그 대표적이다. 특히 화이자 백신을 통한 겨드랑이멍울의 발생사례가 몇 건... ",
"bloggername": "미스터 건강일기",
"bloggerlink": "https://blog.naver.com/msent2287",
"postdate": "20211002"

},
{
"title": "<b>코로나 백신</b>후 샤워 하면 안되는 이유",
"link": "https://ralf79.tistory.com/entry/%EC%BD%94%EB%A1%9C%EB%82%98-%EB%B0%B1%EC%8B%A0%ED%9B%84-%EC%83%A4%EC%9B%8C-%ED%95%98%EB%A9%B4-%EC%95%88%EB%90%98%EB%8A%94-%EC%9D%B4%EC%9C%A0",
"description": "특히 60세에서 74세의 코로나 19 예방접종 예약률은 77.7%를 넘었다. 또한 최근 젊은 연령층들의 접송 비율도 급속도로 빨라지고있다. 화이자, 아스트라제네카등의 백신도 수급이 빨라지고있다. <b>코로나 백신</b>후... ",
"bloggername": "뉴스브런치",
"bloggerlink": "http

get_request_url, [2022-03-15 02:33:39.067360] Url Request Success
<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:39 +0900",
"total": 1470715,
"start": 801,
"display": 100,
"items": [
{
"title": "[후기] <b>코로나백신</b>_화이자2차접종(미열, 근육통, 팔통증)",
"link": "https:\/\/blog.naver.com\/wnguscjsw?Redirect=Log&logNo=222602537230",
"description": "[후기] <b>코로나백신</b>_화이자2차접종 미열, 근육통, 팔통증 지난 화요일, 화이자 2차백신을 맞았다. 남들은... 하루빨리 코로나가 끝나서 마스크없는 세상이 왔으면 싶다. 특히 우리아기들이 마스크없이 뛰어... ",
"bloggername": "ᴢᴏᴏʜᴜɴɪ ෆ",
"bloggerlink": "https://blog.naver.com/wnguscjsw",
"postdate": "20211223"

},
{
"title": "자가면역질환 <b>코로나 백신</b> 2차 후기 | 화이자",
"link": "https:\/\/blog.naver.com\/bubblegumpoo?Redirect=Log&logNo=222492559746",
"description": "드디어 <b>코로나</b> #<b>백신</b>2차 접종까지 맞았다. 1차 보다 훨씬 힘들었던 #화이자 2차 접종 후기 바로 스따뚜. 1차 접종 후기: https://blog.naver.com/bubblegumpoo/222451200345 접종 스케줄 1차 접종: 7/27 오후 3시 2차 접종: 8/30... ",
"bloggername": "버블이의 일상",
"bloggerlink": "https://blog.naver.com/bubblegumpoo",
"po

get_request_url, [2022-03-15 02:33:39.612267] Url Request Success
<getNaverSearchResult, retData > =  {
"lastBuildDate": "Tue, 15 Mar 2022 02:33:39 +0900",
"total": 1470715,
"start": 901,
"display": 100,
"items": [
{
"title": "<b>코로나 백신</b> 3차 접종 (화이자, 당일)",
"link": "https:\/\/blog.naver.com\/gmldud0916?Redirect=Log&logNo=222657569761",
"description": "179일차에 3차 부스터 접종함. 증상이 없어서 종료 #<b>코로나백신</b> #교차접종 #화이자 #아스트라 #2차접종 #1차접종 #<b>코로나백신</b>접종후기 #접종후기 #백신증상 #3차접종 #부스터",
"bloggername": "희바리의 포텐터질기미",
"bloggerlink": "https://blog.naver.com/gmldud0916",
"postdate": "20220225"

},
{
"title": "<b>코로나</b> 감염 입원자의 60%가 2차 접종 후 2주 이상 지난 <b>백신</b>... ",
"link": "https:\/\/blog.naver.com\/peace_2000?Redirect=Log&logNo=222528159921",
"description": "8월 7일 주간에 코로나 감염으로 병원에 입원한 환자들의 60%가 <b>코로나 백신</b> 2차 접종을 완료한지 14일 이상 지난 사람들이었습니다. 8월 21일 주간에는 코로나 감염자의 71%가 백신 접종... ",
"bloggername": "관심가는 대로",
"bloggerlink": "https://blog.naver.com/peace_2000",
"postdate": "20211006"

},
{
"title": "●<b>코로나 백신</b>접종,